In [1]:
import sys
sys.path.append('../src')

In [2]:
import argparse
import math
import os
import time
from functools import partial

import numpy as np
import torch
import visdom

import pyro
import pyro.contrib.examples.multi_mnist as multi_mnist
import pyro.optim as optim
import pyro.poutine as poutine
from components.AIR import AIR, latents_to_tensor
from pyro.contrib.examples.util import get_data_directory
from pyro.infer import SVI, JitTraceGraph_ELBO, TraceGraph_ELBO
from utils.viz import draw_many, tensor_to_objs
from utils.visualizer import plot_mnist_sample
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment
%matplotlib inline
import pandas as pd

## Args

In [3]:
parser = argparse.ArgumentParser(description="Pyro AIR example", argument_default=argparse.SUPPRESS)
parser.add_argument('-n', '--num-steps', type=int, default=int(1e8),
                    help='number of optimization steps to take')
parser.add_argument('-b', '--batch-size', type=int, default=64,
                    help='batch size')
parser.add_argument('-lr', '--learning-rate', type=float, default=1e-4,
                    help='learning rate')
parser.add_argument('-blr', '--baseline-learning-rate', type=float, default=1e-3,
                    help='baseline learning rate')
parser.add_argument('--progress-every', type=int, default=1,
                    help='number of steps between writing progress to stdout')
parser.add_argument('--eval-every', type=int, default=0,
                    help='number of steps between evaluations')
parser.add_argument('--baseline-scalar', type=float,
                    help='scale the output of the baseline nets by this value')
parser.add_argument('--no-baselines', action='store_true', default=False,
                    help='do not use data dependent baselines')
parser.add_argument('--encoder-net', type=int, nargs='+', default=[200],
                    help='encoder net hidden layer sizes')
parser.add_argument('--decoder-net', type=int, nargs='+', default=[200],
                    help='decoder net hidden layer sizes')
parser.add_argument('--predict-net', type=int, nargs='+',
                    help='predict net hidden layer sizes')
parser.add_argument('--embed-net', type=int, nargs='+',
                    help='embed net architecture')
parser.add_argument('--bl-predict-net', type=int, nargs='+',
                    help='baseline predict net hidden layer sizes')
parser.add_argument('--non-linearity', type=str,
                    help='non linearity to use throughout')
parser.add_argument('--viz', action='store_true', default=True,
                    help='generate vizualizations during optimization')
parser.add_argument('--viz-every', type=int, default=100,
                    help='number of steps between vizualizations')
parser.add_argument('--visdom-env', default='main',
                    help='visdom enviroment name')
parser.add_argument('--load', type=str, default="/Users/chamathabeysinghe/Projects/monash/VAE_v2/checkpoints/model-size-75-3ants.ckpt",
                    help='load previously saved parameters')
parser.add_argument('--save', type=str, default="/Users/chamathabeysinghe/Projects/monash/VAE_v2/checkpoints/model-size-75-3ants.ckpt",
                    help='save parameters to specified file')
parser.add_argument('--save-every', type=int, default=100,
                    help='number of steps between parameter saves')
parser.add_argument('--cuda', action='store_true', default=False,
                    help='use cuda')
parser.add_argument('--jit', action='store_true', default=False,
                    help='use PyTorch jit')
parser.add_argument('-t', '--model-steps', type=int, default=3,
                    help='number of time steps')
parser.add_argument('--rnn-hidden-size', type=int, default=256,
                    help='rnn hidden size')
parser.add_argument('--encoder-latent-size', type=int, default=50,
                    help='attention window encoder/decoder latent space size')
parser.add_argument('--decoder-output-bias', type=float,
                    help='bias added to decoder output (prior to applying non-linearity)')
parser.add_argument('--decoder-output-use-sigmoid', action='store_true',
                    help='apply sigmoid function to output of decoder network')
parser.add_argument('--window-size', type=int, default=28,
                    help='attention window size')
parser.add_argument('--z-pres-prior', type=float, default=0.5,
                    help='prior success probability for z_pres')
parser.add_argument('--z-pres-prior-raw', action='store_true', default=False,
                    help='use --z-pres-prior directly as success prob instead of a geometric like prior')
parser.add_argument('--anneal-prior', choices='none lin exp'.split(), default='none',
                    help='anneal z_pres prior during optimization')
parser.add_argument('--anneal-prior-to', type=float, default=1e-7,
                    help='target z_pres prior prob')
parser.add_argument('--anneal-prior-begin', type=int, default=0,
                    help='number of steps to wait before beginning to anneal the prior')
parser.add_argument('--anneal-prior-duration', type=int, default=100000,
                    help='number of steps over which to anneal the prior')
parser.add_argument('--pos-prior-mean', type=float,
                    help='mean of the window position prior')
parser.add_argument('--pos-prior-sd', type=float,
                    help='std. dev. of the window position prior')
parser.add_argument('--scale-prior-mean', type=float,
                    help='mean of the window scale prior')
parser.add_argument('--scale-prior-sd', type=float,
                    help='std. dev. of the window scale prior')
parser.add_argument('--no-masking', action='store_true', default=False,
                    help='do not mask out the costs of unused choices')
parser.add_argument('--seed', type=int, help='random seed', default=None)
parser.add_argument('-v', '--verbose', action='store_true', default=False,
                    help='write hyper parameters and network architecture to stdout')

_StoreTrueAction(option_strings=['-v', '--verbose'], dest='verbose', nargs=0, const=True, default=False, type=None, choices=None, help='write hyper parameters and network architecture to stdout', metavar=None)

In [4]:
# vars(parser.parse_args(""))
args = argparse.Namespace(**vars(parser.parse_args("")))

In [8]:
def make_prior(k):
    assert 0 < k <= 1
    u = 1 / (1 + k + k**2 + k**3)
    p0 = 1 - u
    p1 = 1 - (k * u) / p0
    p2 = 1 - (k**2 * u) / (p0 * p1)
    trial_probs = [p0, p1, p2]
    # dist = [1 - p0, p0 * (1 - p1), p0 * p1 * (1 - p2), p0 * p1 * p2]
    # print(dist)
    return lambda t: trial_probs[t]

In [9]:
# if 'save' in args:
#     if os.path.exists(args.save):
#         raise RuntimeError('Output file "{}" already exists.'.format(args.save))

if args.seed is not None:
    pyro.set_rng_seed(args.seed)

# Build a function to compute z_pres prior probabilities.
if args.z_pres_prior_raw:
    def base_z_pres_prior_p(t):
        return args.z_pres_prior
else:
    base_z_pres_prior_p = make_prior(args.z_pres_prior)

# Wrap with logic to apply any annealing.
def z_pres_prior_p(opt_step, time_step):
    p = base_z_pres_prior_p(time_step)
    if args.anneal_prior == 'none':
        return p
    else:
        decay = dict(lin=lin_decay, exp=exp_decay)[args.anneal_prior]
        return decay(p, args.anneal_prior_to, args.anneal_prior_begin,
                     args.anneal_prior_duration, opt_step)



## Data loader

In [14]:
def load_data(base_path):
    path = base_path + '/{:05d}.png'
    X_np = []
    for i in range(SAMPLE_COUNT):      
        img = np.asarray(Image.open(path.format(i)))
        X_np.append(img)
        
    X_np = np.asarray(X_np)    
    X_np = X_np.astype(np.float32)
    X_np /= 255.0
    X = torch.from_numpy(X_np)
    # Using FloatTensor to allow comparison with values sampled from
    # Bernoulli.
    counts = torch.FloatTensor([1 for objs in X_np])
    return X, counts



## Model

In [5]:
model_arg_keys = ['window_size',
                  'rnn_hidden_size',
                  'decoder_output_bias',
                  'decoder_output_use_sigmoid',
                  'baseline_scalar',
                  'encoder_net',
                  'decoder_net',
                  'predict_net',
                  'embed_net',
                  'bl_predict_net',
                  'non_linearity',
                  'pos_prior_mean',
                  'pos_prior_sd',
                  'scale_prior_mean',
                  'scale_prior_sd']
model_args = {key: getattr(args, key) for key in model_arg_keys if key in args}

In [ ]:
air = AIR(
        num_steps=args.model_steps,
        x_size=75,
        use_masking=not args.no_masking,
        use_baselines=not args.no_baselines,
        z_what_size=args.encoder_latent_size,
        use_cuda=args.cuda,
        **model_args
    )

In [ ]:
if 'load' in args:
    print('Loading parameters...')
    air.load_state_dict(torch.load(args.load))

## Visualize

In [ ]:
count = 100
vis = visdom.Visdom(env=args.visdom_env)

## Post processing

In [ ]:
def img2arr(img):
    # assumes color image
    # returns an array suitable for sending to visdom
#     return np.asarray(img)
    return np.array(img.getdata(), np.uint8).reshape(img.size + (3,)).transpose((2, 0, 1))

def arr2img(arr):
    # arr is expected to be a 2d array of floats in [0,1]
    return Image.frombuffer('L', arr.shape, (arr * 255).astype(np.uint8).tostring(), 'raw', 'L', 0, 1)

def bounding_box(z_where, x_size):
    """This doesn't take into account interpolation, but it's close
    enough to be usable."""
    w = x_size / z_where.s
    h = x_size / z_where.s
    xtrans = -z_where.x / z_where.s * x_size / 2.
    ytrans = -z_where.y / z_where.s * x_size / 2.
    x = (x_size - w) / 2 + xtrans  # origin is top left
    y = (x_size - h) / 2 + ytrans
    return (x, y), w, h

def colors(k):
    return [(255, 0, 0), (0, 255, 0), (0, 0, 255)][k % 3]

In [ ]:
def draw_one(imgarr, z_arr):
    # Note that this clipping makes the visualisation somewhat
    # misleading, as it incorrectly suggests objects occlude one
    # another.
    clipped = np.clip(imgarr.detach().cpu().numpy(), 0, 1)
    img = arr2img(clipped).convert('RGB')
    draw = ImageDraw.Draw(img)
    for k, z in enumerate(z_arr):
        # It would be better to use z_pres to change the opacity of
        # the bounding boxes, but I couldn't make that work with PIL.
        # Instead this darkens the color, and skips boxes altogether
        # when z_pres==0.
        if z.pres > 0:
            (x, y), w, h = bounding_box(z, imgarr.size(0))
            color = tuple(map(lambda c: int(c * z.pres), colors(k)))
            crop_img = clipped[int(y):int(y)+int(h), int(x):int(x)+int(w)]
            white_count = np.count_nonzero(crop_img>0.01)
            black_count = np.count_nonzero(crop_img<0.01)
            if (black_count > 0 and white_count / black_count < 0.09):
                continue
            draw.rectangle([x, y, x + w, y + h], outline=color)
    is_relaxed = any(z.pres != math.floor(z.pres) for z in z_arr)
    fmtstr = '{:.1f}' if is_relaxed else '{:.0f}'
    draw.text((0, 0), fmtstr.format(sum(z.pres for z in z_arr)), fill='white')
    return img2arr(img)

In [ ]:
def draw_many_custom(imgarrs, z_arr):
    # canvases is expected to be a (n,w,h) numpy array
    # z_where_arr is expected to be a list of length n
    return [draw_one(imgarr, z) for (imgarr, z) in zip(imgarrs.cpu(), z_arr)]


In [ ]:
examples_to_viz = X[0:50]
trace = poutine.trace(air.guide).get_trace(examples_to_viz, None)
z, recons = poutine.replay(air.prior, trace=trace)(examples_to_viz.size(0))
z_wheres = tensor_to_objs(latents_to_tensor(z))

# Show data with inferred objection positions.
vis.images(draw_many_custom(examples_to_viz, z_wheres))
# Show reconstructions of data.
vis.images(draw_many(examples_to_viz, z_wheres))

## Better post processing

In [ ]:
def draw_bboxes_img(img, bboxes):
    img = Image.fromarray((img*255).astype(np.uint8), mode='L').convert('RGB')
    draw = ImageDraw.Draw(img)
    color = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
    for i, bbox in enumerate(bboxes):
        x, y, w, h = bbox
        draw.rectangle([x, y, x + w, y + h], outline=color[i%3])
    return np.asarray(img)
        

In [ ]:
def filter_by_bw_ratio(img, bboxes, box_stds):
    selected_bboxes = []
    selected_stds = []
    selected_bw_ratios = []
    for i in range(len(bboxes)):
        bbox = bboxes[i]
        x, y, w, h = bbox
        clipped = np.clip(img, 0, 1)
        crop_img = clipped[int(y):int(y)+int(h), int(x):int(x)+int(w)]
        white_count = np.count_nonzero(crop_img>0.01)
        black_count = np.count_nonzero(crop_img<0.01)
        if (black_count > 0 and white_count / black_count < 0.09):
            continue
        selected_bboxes.append(bboxes[i])
        selected_stds.append(box_stds[i])
        selected_bw_ratios.append(white_count / max(black_count, .00001))
    return selected_bboxes, selected_stds, selected_bw_ratios


In [ ]:
def get_bounding_boxes_for_image(img, z_arr):
    bounding_boxes = []
    for k, z in enumerate(z_arr):
        if z.pres > 0:
            (x, y), w, h = bounding_box(z, img.shape[0])
            x, y, w, h = x.item(), y.item(), w.item(), h.item()
            bounding_boxes.append([x, y, w, h])
    return bounding_boxes
            

In [ ]:
def multiple_predict(img):
    img_array = np.asarray([np.copy(img) for _ in range(REPEAT_COUNT)])
    img_array = torch.from_numpy(img_array)
    
    trace = poutine.trace(air.guide).get_trace(img_array, None)
    z, recons = poutine.replay(air.prior, trace=trace)(img_array.size(0))
    z_wheres = tensor_to_objs(latents_to_tensor(z))
    bboxes_frame =  []
    for counter, z in enumerate(z_wheres):
        bboxes = get_bounding_boxes_for_image(img, z)
        img2 = draw_bboxes_img(img, bboxes)
        bboxes_frame.append(bboxes)
#         plt.imshow(img2)
#         plt.show()
    return bboxes_frame


In [ ]:
def match_boxes(bboxes_frame):
    parents = [[-1, -1, -1] for _ in range(REPEAT_COUNT)]
    original_boxes = [] # [{parent: [], childs: [[]]}]

    for frame_index in range(0, REPEAT_COUNT):
        current_boxes = bboxes_frame[frame_index]

        for box_index in range(len(current_boxes)):
            if (parents[frame_index][box_index] == -1):
                original_boxes.append({'parent': current_boxes[box_index], 'childs': []})

        if (frame_index + 1 >= REPEAT_COUNT):
            break

        future_boxes = bboxes_frame[frame_index + 1]

        roi_f_box_orginal_box = [[0 for _ in range(len(original_boxes))] for _ in range(len(future_boxes))]
        for f_box_index in range(len(future_boxes)):
            for o_box_index in range(len(original_boxes)):
                bb_f = future_boxes[f_box_index]
                bb_o = original_boxes[o_box_index]['parent']

                roi_f_box_orginal_box[f_box_index][o_box_index] = get_iou(
                    {'x1': bb_f[0] ,'x2': bb_f[0] + bb_f[2] ,'y1': bb_f[1] ,'y2': bb_f[1]+bb_f[3]},
                    {'x1': bb_o[0] ,'x2': bb_o[0] + bb_o[2] ,'y1': bb_o[1] ,'y2': bb_o[1]+bb_o[3]}
                )

    #     for f_box_index in range(len(future_boxes)):
    #         print(roi_f_box_orginal_box[f_box_index])
        roi_f_box_orginal_box = np.asarray(roi_f_box_orginal_box) * -1
        row_ind, col_ind = linear_sum_assignment(roi_f_box_orginal_box)

        for i in range(len(row_ind)):
            f_f_index = row_ind[i]
            o_f_index = col_ind[i]
            parents[frame_index+1][f_f_index] = o_f_index
            original_boxes[o_f_index]['childs'].append(future_boxes[f_f_index])
        
        return original_boxes

In [ ]:
def calculate_mean_std(original_boxes):
    box_means = []
    box_stds = []
    for i, item in enumerate(original_boxes):
        parent = item['parent']
        boxes = item['childs']
        boxes.append(parent)
        boxes = np.asarray(boxes)
        box_means.append(np.mean(boxes, axis=0))
        box_stds.append(np.std(boxes, axis=0))
    return box_means, box_stds


In [ ]:
def filter_by_std(box_means, box_stds):
    selected_boxes = []
    selected_stds = []
    for i in range(len(box_means)):
        #Todo improve this statement
        selected = True
        for j in range(4):
            if (box_stds[i][j] >= STD_THRESHOLD):
                selected = False
                break

        if selected:
            selected_boxes.append(box_means[i])
            selected_stds.append(box_stds[i])

    return selected_boxes, selected_stds

In [ ]:
def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    
    assert bb1['x1'] <= bb1['x2']
    assert bb1['y1'] <= bb1['y2']
    assert bb2['x1'] <= bb2['x2']
    assert bb2['y1'] <= bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0    
    
    
    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box.
    # NOTE: We MUST ALWAYS add +1 to calculate area when working in
    # screen coordinates, since 0,0 is the top left pixel, and w-1,h-1
    # is the bottom right pixel. If we DON'T add +1, the result is wrong.
    intersection_area = (x_right - x_left + 1) * (y_bottom - y_top + 1)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1'] + 1) * (bb1['y2'] - bb1['y1'] + 1)
    bb2_area = (bb2['x2'] - bb2['x1'] + 1) * (bb2['y2'] - bb2['y1'] + 1)
    
    
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [ ]:
def predict_bboxes(img):
    bboxes_frame = multiple_predict(img)
    original_boxes = match_boxes(bboxes_frame)
    box_means, box_stds = calculate_mean_std(original_boxes)
    box_means, box_stds = filter_by_std(box_means, box_stds)
    
    img2 = draw_bboxes_img(img, box_means)

In [ ]:
def run_test():
    images = []
    images_gt = []
    csv_predictions = []
    for i in range(SAMPLE_COUNT):
        img = examples_to_viz.detach().cpu().numpy()[i]
        bboxes_frame = multiple_predict(img)
        original_boxes = match_boxes(bboxes_frame)
        selected_boxes, box_stds = calculate_mean_std(original_boxes)
        selected_boxes, box_stds, selected_bw_ratios = filter_by_bw_ratio(img, selected_boxes, box_stds)
#         selected_boxes, box_stds = filter_by_std(selected_boxes, box_stds)
        img2 = draw_bboxes_img(img, selected_boxes)
        images.append(img2)
        
        
        gt_boxes = Y_df.loc[Y_df['frame'] == i][['y', 'x', 'w', 'h']]
        img2_gt = draw_bboxes_img(img, gt_boxes.values)
        images_gt.append(img2_gt)
        
        predictions = list(map(lambda x: [i]+ list(x[0]) + [x[1]] , zip(selected_boxes, selected_bw_ratios)))
        csv_predictions += predictions

    vis_images = [img2arr(Image.fromarray(img)) for img in images]
    vis.images(vis_images)
    
    vis_images_gt = [img2arr(Image.fromarray(img)) for img in images_gt]
    vis.images(vis_images_gt)
    return csv_predictions
    


In [11]:
ant_folder = 3
dataset_type = 'simple_dataset'
base_path = "/Users/chamathabeysinghe/Projects/monash/VAE_v2/data/synthetic/{}/original:_300-resize:_75-{}_ants".format(dataset_type, ant_folder)
SAMPLE_COUNT = 100


X, counts = load_data(base_path)
X_size = X.size(0)
if args.cuda:
    X = X.cuda()
    
data_csv = base_path + ".csv"
Y_df = pd.read_csv(data_csv)
    

In [16]:
X.shape

torch.Size([100, 75, 75])

In [ ]:
REPEAT_COUNT = 50
BOX_COUNT = 3
STD_THRESHOLD = 2
examples_to_viz = X[0:SAMPLE_COUNT]

predictions = run_test()


predictions_df = pd.DataFrame(np.asarray(predictions), columns=['frame', 'y', 'x', 'w', 'h', 'score'])
predictions_df = predictions_df[['frame', 'x', 'y', 'h', 'w', 'score']]
file_name = base_path.split('/')[-1]
save_path = '/Users/chamathabeysinghe/Projects/monash/VAE_v2/predictions/{}/{}.csv'.format(dataset_type, file_name)
predictions_df.to_csv(save_path, index=None)
